In [ ]:
using LinearAlgebra
import Optim as opt

In [ ]:
T       = [1 1/2; 0 √3/2];
Gm       = 2π*inv(T)';
Rk      = 7

Rk_norm = (x,y) -> (sign(x) == - sign(y)) ? abs(x-y) : max(abs(x),abs(y))
k_mesh  = [[i,j] for i in -Rk:Rk for j in -Rk:Rk if Rk_norm(i,j)<=Rk];
Nk      = length(k_mesh)
Ks      = [Gm*v for v in k_mesh];

knorm    = v -> norm(Gm * v)

Coulomb = q -> (q==0 ? 0. : 2π/det(T)/q);

base_kinetic  = knorm.(k_mesh) .^2/2;
qnorms        = [knorm(k1-k2) for k1 in k_mesh, k2 in k_mesh]

base_Coulomb = Coulomb.(qnorms);

In [ ]:
function scenario_constructor(L, M, d)    
    KEs = base_kinetic/L^2/M
    
    intralayer = base_Coulomb/L
    
    interlayer = intralayer .* exp.(-qnorms *d/L)
    
    println((sum(KEs) - sum(intralayer))/(2 * Nk))
    
    function energy(thetas, µ)
        occupations = sin.(thetas/2) .^2
        sintheta = sin.(thetas)/2
        
        KE = dot(KEs .- 2µ, occupations)
        
        fock = (occupations' * intralayer) * occupations
        hartree = (sintheta' * interlayer) * sintheta
        
        return KE - fock - hartree
    end
    
    function energy_grad!(G, thetas, µ)
        occupations = sin.(thetas/2) .^2
        sintheta = sin.(thetas)/2
        
        dd_occupations = sintheta
        dd_sintheta = cos.(thetas)/2
        
        dd_KE = (KEs .- 2µ) .* dd_occupations
        
        dd_fock = 2 * (intralayer * occupations) .* dd_occupations
        dd_hartree = 2 * (interlayer * sintheta) .* dd_sintheta
        
        copyto!(G, dd_KE - dd_fock - dd_hartree)
    end
    
    return energy, energy_grad!, L^2 * det(T), Nk, KEs
end

In [ ]:
m = (1 ./ LinRange(2., 8., 10))[5]
d = 1
Ns = 0.026 * LinRange(0.1, 1.3, 30)


ind = 1
L = sqrt(32/det(T)/Ns[ind])

energy, energy_grad!, A, Nk, KEs = scenario_constructor(L, m/(1+m), d);

In [ ]:
µs = LinRange(-0.04, 0.0, 99)[end:-1:1]

points = []

for µ in µs
    E = x -> energy(x, µ)
    g! = (G, x) -> energy_grad!(G, x, µ)

    start = π/2 * exp.(-3*KEs/KEs[1]);

    optim_options = opt.Options(time_limit=20, f_reltol=1e-5);
    sol = opt.optimize(E, g!, start, opt.ConjugateGradient(),optim_options)

    occupations = sin.(sol.minimizer/2).^2
    N = sum(occupations)
    
    push!(points, (N/A, sol.minimum/N/2+µ))
end